In [3]:
from utils.checkpoints import load_checkpoint
from constants import *
from models.densenet121 import DenseNet121_change_avg
from dataloader.dataloader import HmDataset
from dataloader.transforms import transforms
from main.fit import fit

from torch.utils.data import DataLoader
import torch.optim as optim
import torch.nn as nn

# get test data loader
train_dataset = HmDataset(df_path='./dataset/train.csv', transforms=transforms, mode=DATASET_MODE)
valid_dataset = HmDataset(df_path='./dataset/valid.csv', transforms=transforms, mode=DATASET_MODE)
test_dataset = HmDataset(df_path='./dataset/test.csv', transforms=transforms, mode=DATASET_MODE)

train_loader = DataLoader(train_dataset,
                         batch_size=1,
                         shuffle=False,
                         num_workers=4)
valid_loader = DataLoader(valid_dataset,
                         batch_size=1,
                         shuffle=False,
                         num_workers=4)
test_loader = DataLoader(test_dataset,
                         batch_size=1,
                         shuffle=False,
                         drop_last=False,
                         num_workers=4)

# create model
model = DenseNet121_change_avg()
criterion = nn.BCELoss()
optimizer = optim.SGD(model.parameters(), lr=INITIAL_LR, momentum=0.9)

# load model
model =  DenseNet121_change_avg()
model, optimizer, _, epoch = load_checkpoint('./checkpoints/200601_223249_DenseNet121_LR0.001_BS64_BCELoss/  8.pth', model, optimizer)

device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.to(device)
''''''

''

In [131]:
from tqdm import tqdm
import pandas as pd
def get_features(data_loader):
    
    for i, (filename, target, inputs) in enumerate(tqdm(data_loader, position=0)):

        target, inputs = target.to(device), inputs.to(device)
        pred_label, features = model(inputs)
        
        df_filename = pd.DataFrame([filename], columns=['filename'])
        
        df_label = pd.DataFrame(np.expand_dims(pred_label.squeeze().cpu().detach().numpy(), axis=0))
        df_label = df_label.add_prefix('label_')
        
        df_features = pd.DataFrame(np.expand_dims(features.squeeze().cpu().detach().numpy(), axis=0))
        df_features = df_features.add_prefix('feature_')
        
        row_df = pd.concat([df_filename, df_label, df_features], axis=1)

        if i==0:
            df = row_df
        else:
            df = df.append(row_df)
            
    return df

In [135]:
df = get_features(train_loader)
df.to_csv('./dataset/train_features.csv', index=None)

100%|██████████| 2760/2760 [02:16<00:00, 20.16it/s]


In [136]:
df = get_features(valid_loader)
df.to_csv('./dataset/valid_features.csv', index=None)

100%|██████████| 336/336 [00:16<00:00, 20.65it/s]


In [132]:
df = get_features(test_loader)
df.to_csv('./dataset/test_features.csv', index=None)

100%|██████████| 333/333 [00:16<00:00, 19.92it/s]
